In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a ")

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=-1,
    pipeline_kwargs={
        "max_new_tokens": 100,
    },
)

chain = prompt | llm

print(chain.invoke({"word": "tomato"}))


ImportError: Could not load the text-generation model due to missing dependencies.